# Bayesian Inference Modelling
## Goal: estimation of both evidence and posterior in one go.

# 1. Matern kernel and Gaussian prior
It is well-known that BQ modelling is limited to certain kernel mean.<br>
The combination of Gaussian prior and Matern kernel has no analytical kernel mean.<br>
We show our BASQ framework can perform BQ for arbitrary kernel and prior distribution via Nyström approximation.<br>
The problems are the same with tutorial 01.

In [1]:
import torch
import time
import gpytorch
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.means import ZeroMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.constraints import Interval
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model

def set_model(X, Y):
    base_kernel = MaternKernel(nu=2.5) # Matern kernel
    mean_module = ZeroMean()
    covar_module = ScaleKernel(base_kernel)

    # Set a GP model
    train_Y = Y.view(-1).unsqueeze(1)
    likelihood = GaussianLikelihood()
    likelihood.noise_covar.register_constraint("raw_noise", gpytorch.constraints.Interval(1e-12, 1e-9))
    model = SingleTaskGP(X, train_Y, likelihood=likelihood, mean_module=mean_module, covar_module=covar_module)
    hypers = {'likelihood.noise_covar.noise': torch.tensor(1e-10)}
    model.initialize(**hypers)
    model.likelihood.raw_noise.requires_grad = False
    return model

def optimise_model(model):
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    return model

def set_and_opt_gp(X, Y):
    model = set_model(X, Y)
    model = optimise_model(model)
    return model

In [2]:
from SOBER._utils import TensorManager
from SOBER.BASQ._basq import BASQ
from BASQ.experiment.gmm import GMM
#import warnings
tm = TensorManager()
#warnings.filterwarnings("ignore")


num_dim = 10  # Number of dimensions of the true likelihood to be estimated
mu_pi = torch.zeros(num_dim).to(tm.device, tm.dtype)  # the mean vactor of Gaussian prior
cov_pi = 2 * torch.eye(num_dim).to(tm.device, tm.dtype)  # the covariance matrix of Gaussian prior

from SOBER._prior import Gaussian
prior = Gaussian(mu_pi, cov_pi)
true_likelihood = GMM(num_dim, mu_pi, cov_pi, tm.device)  # true likelihood to be estimated

In [3]:
n_init = 2           # number of initial guess
n_iterations = 10    # number of iterations (batches)
n_cand = 20000       # number of candidates
n_nys = 500          # number of Nyström samples
n_batch = 100        # batch size

# Run!

In [4]:
Z_true = 1                             # true integral
x_test = prior.sample(10000)           # test data for evaluating posterior using KL divergence

torch.manual_seed(0)                    # fix random seed for reproducibility
X = prior.sample(n_init)               # inital dataset X
Y = true_likelihood(X).to(tm.dtype)    # initial guess Y
basq = BASQ(n_cand, n_nys, prior)      # set up BASQ instance
model = set_and_opt_gp(X, Y)           # set up the GP surroage model

for ith_round in range(n_iterations):
    tik = time.monotonic()
    X_batch, _ = basq.batch_uncertainty_sampling(model, n_batch)  # run BASQ algorithm to select 100 batch points
    tok = time.monotonic()
    overhead = tok - tik               # overhead of batch query
    
    Y_batch = true_likelihood(X_batch) # parallel query to true likelihood function
    X = torch.cat([X, X_batch])        # concatenate the observations for X
    Y = torch.cat([Y, Y_batch])        # concatenate the observations for X
    
    # Evaluation for integral
    model = set_and_opt_gp(X, Y)       # retrain GP model
    integral_estimated = basq.quadrature(model, 500)  # integral estimation
    logMAE = (Z_true - integral_estimated).abs()      # evaluate the estimated integral value to true one
    # EZ, VZ = basq.full_quadrature(model, 500)       # You can estimate integral variance (but takes more time)
    
    # Evaluation for the posterior
    KL = basq.KLdivergence(Z_true, x_test, true_likelihood, model)  # compute the KL divergence
    print('Iter %d - overhead: %.3f [s]  logMAE of Integral: %.3f   logKL of posterior: %.3f' % (
        ith_round, overhead, logMAE.log().item(), KL.log().item()
    ))

Iter 0 - overhead: 0.954 [s]  logMAE of Integral: -1.825   logKL of posterior: -8.284
Iter 0 - overhead: 0.954 [s]  logMAE of Integral: -1.825   logKL of posterior: -8.284
Iter 1 - overhead: 0.932 [s]  logMAE of Integral: -2.822   logKL of posterior: -9.585
Iter 1 - overhead: 0.932 [s]  logMAE of Integral: -2.822   logKL of posterior: -9.585
Iter 2 - overhead: 2.116 [s]  logMAE of Integral: -2.875   logKL of posterior: -9.697
Iter 2 - overhead: 2.116 [s]  logMAE of Integral: -2.875   logKL of posterior: -9.697
Iter 3 - overhead: 1.595 [s]  logMAE of Integral: -2.153   logKL of posterior: -8.630
Iter 3 - overhead: 1.595 [s]  logMAE of Integral: -2.153   logKL of posterior: -8.630
Iter 4 - overhead: 1.546 [s]  logMAE of Integral: -2.695   logKL of posterior: -8.812
Iter 4 - overhead: 1.546 [s]  logMAE of Integral: -2.695   logKL of posterior: -8.812
Iter 5 - overhead: 1.789 [s]  logMAE of Integral: -3.358   logKL of posterior: -9.171
Iter 5 - overhead: 1.789 [s]  logMAE of Integral: -3.3

# 2. Spectral Mixture kernel and Gaussian prior
Next, we will try spectral mixture kernel, which is also popular kernel.

In [5]:
import gpytorch

class SpectralMixtureGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(SpectralMixtureGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.SpectralMixtureKernel(num_mixtures=4, ard_num_dims=num_dim)
        self.covar_module.initialize_from_data(train_x, train_y)

    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def set_model(X, Y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = SpectralMixtureGPModel(X, Y, likelihood)
    likelihood.noise_covar.register_constraint("raw_noise", gpytorch.constraints.Interval(1e-12, 1e-9))
    hypers = {'likelihood.noise_covar.noise': torch.tensor(1e-10)}
    model.initialize(**hypers)
    model.likelihood.raw_noise.requires_grad = False
    return model

def optimise_model(model, X, Y):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

    for i in range(100):
        optimizer.zero_grad()
        output = model(X)
        loss = -mll(output, Y)
        loss.backward()
        optimizer.step()
    return model

def set_and_opt_gp(X, Y):
    model = set_model(X, Y)
    model = optimise_model(model, X, Y)
    return model

In [6]:
n_init = 2           # number of initial guess
n_iterations = 10    # number of iterations (batches)
n_cand = 10000       # number of candidates
n_nys = 100          # number of Nyström samples
n_batch = 100        # batch size

In [ ]:
Z_true = 1                             # true integral
x_test = prior.sample(10000)           # test data for evaluating posterior using KL divergence

torch.manual_seed(0)                    # fix random seed for reproducibility
X = prior.sample(n_init)               # inital dataset X
Y = true_likelihood(X).to(tm.dtype)    # initial guess Y
basq = BASQ(n_cand, n_nys, prior)      # set up BASQ instance
model = set_and_opt_gp(X, Y)           # set up the GP surroage model

for ith_round in range(n_iterations):
    tik = time.monotonic()
    X_batch, _ = basq.batch_uncertainty_sampling(model, n_batch)  # run BASQ algorithm to select 100 batch points
    tok = time.monotonic()
    overhead = tok - tik               # overhead of batch query
    
    Y_batch = true_likelihood(X_batch) # parallel query to true likelihood function
    X = torch.cat([X, X_batch])        # concatenate the observations for X
    Y = torch.cat([Y, Y_batch])        # concatenate the observations for X
    
    # Evaluation for integral
    model = set_and_opt_gp(X, Y)       # retrain GP model
    integral_estimated = basq.quadrature(model, 100)  # integral estimation
    logMAE = (Z_true - integral_estimated).abs()      # evaluate the estimated integral value to true one
    # EZ, VZ = basq.full_quadrature(model, 500)       # You can estimate integral variance (but takes more time)
    
    # Evaluation for the posterior
    KL = basq.KLdivergence(Z_true, x_test, true_likelihood, model)  # compute the KL divergence
    print('Iter %d - overhead: %.3f [s]  logMAE of Integral: %.3f   logKL of posterior: %.3f' % (
        ith_round, overhead, logMAE.log().item(), KL.log().item()
    ))

Iter 0 - overhead: 1.759 [s]  logMAE of Integral: 0.000   logKL of posterior: -4.910
Iter 0 - overhead: 1.759 [s]  logMAE of Integral: 0.000   logKL of posterior: -4.910
Iter 1 - overhead: 3.045 [s]  logMAE of Integral: 0.000   logKL of posterior: -2.329
Iter 1 - overhead: 3.045 [s]  logMAE of Integral: 0.000   logKL of posterior: -2.329
Iter 2 - overhead: 3.267 [s]  logMAE of Integral: 0.000   logKL of posterior: -0.584
Iter 2 - overhead: 3.267 [s]  logMAE of Integral: 0.000   logKL of posterior: -0.584
Iter 3 - overhead: 5.185 [s]  logMAE of Integral: -0.000   logKL of posterior: -4.794
Iter 3 - overhead: 5.185 [s]  logMAE of Integral: -0.000   logKL of posterior: -4.794
Iter 4 - overhead: 6.836 [s]  logMAE of Integral: -0.000   logKL of posterior: -3.105
Iter 4 - overhead: 6.836 [s]  logMAE of Integral: -0.000   logKL of posterior: -3.105
Iter 5 - overhead: 17.360 [s]  logMAE of Integral: -0.000   logKL of posterior: -4.014
Iter 5 - overhead: 17.360 [s]  logMAE of Integral: -0.000  